In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize
from selenium import webdriver
import time

# 기업코드 가져오기

In [12]:
def show_me_the_corp_code(days = 5):
    driver = webdriver.Chrome("C:/Users/elris/Downloads/chromedriver_win32 (1)/chromedriver.exe")
    url = 'https://dart.fss.or.kr/dsac001/mainY.do#'
    driver.get(url)
    ls = []
    
    for j in range(2,days+1): 
        driver.find_element_by_css_selector('#date > p > a:nth-child({})'.format(j)).click()            
        time.sleep(1)
        for _ in range(2):
            for i in range(1,101):
                try : 
                    driver.execute_script('window.scrollTo(10,{});'.format(i*100))
                    driver.find_element_by_css_selector('#listContents > div.table_list > table > tbody > tr:nth-child({}) > td:nth-child(2) > span > a'.format(i)).click()
                    time.sleep(1)
                    ls.append(driver.find_element_by_css_selector('#pop_body > div > table > tbody > tr:nth-child(4) > td').text)
                    time.sleep(1)
                    driver.find_element_by_css_selector('#ext-gen48').click()
                except : pass
            try :
                driver.find_element_by_css_selector('#listContents > div.page_list > input[type="button"]:nth-child(4)').click()
            except : pass

    driver.find_element_by_css_selector('#date > p > a:nth-child(1)').click()
    for i in range(101):
        try : 
            driver.execute_script('window.scrollTo(10,{});'.format(i*100))
            driver.find_element_by_css_selector('#listContents > div.table_list > table > tbody > tr:nth-child({}) > td:nth-child(2) > span > a'.format(i)).click()
            time.sleep(1)
            ls.append(driver.find_element_by_css_selector('#pop_body > div > table > tbody > tr:nth-child(4) > td').text)
            time.sleep(1)
            driver.find_element_by_css_selector('#ext-gen48').click()
        except : pass

    driver.close()
    return ls

In [13]:
%time code_ls = show_me_the_corp_code(2)

Wall time: 6min 6s


In [14]:
len(code_ls)

153

In [15]:
code_ls_new = list(set(code_ls))
code_ls_new = [i for i in code_ls_new if i is not '']
len(code_ls_new)

61

In [16]:
import pandas as pd
df = pd.DataFrame(data=code_ls_new,columns=['code'])
df.to_csv('corp_code.csv',index=False)

In [17]:
code_df = pd.read_csv('corp_code.csv')
code_list = list(code_df['code'])

In [18]:
code_list = [str(code) for code in code_list]

In [19]:
code_list = ['0'*(6-len(code))+code for code in code_list if len(code) != 6]

In [20]:
code_list[0:10]

['030610',
 '003070',
 '002020',
 '009540',
 '008560',
 '003470',
 '016610',
 '006800',
 '001510',
 '005940']

# 여기서 부터는 실행이 안됌(어떤 형식인지 모르겠음)

In [21]:
# 회사의 전체 사업+반기+분기보고서 100건
url_tmpl = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set={page}'
auth = '7929e9e6b73745a684d56abb329dd353b9c4a5bd'
for i,code in enumerate(code_list):
    page = '100'
    url = url_tmpl.format(auth=auth,code=code,page=page)
    r = requests.get(url)
    jo = json.loads(r.text)
    globals()['result'+str(i)] = json_normalize(jo, 'list')

In [22]:
df = result[:] 
for ix, row in df.iterrows():
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(row['rcp_no'])
    print (ix, url)

NameError: name 'result' is not defined

In [23]:
industrial_ls = [result['rcp_no'][i] for i in range(len(result)) if int(result['rcp_no'][i][5])%3 == 0]

NameError: name 'result' is not defined

In [24]:
industrial_url = []
for num in industrial_ls:
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(num)
    industrial_url.append(url)
industrial_url

NameError: name 'industrial_ls' is not defined

# url

In [25]:
target_url = 'http://dart.fss.or.kr/report/viewer.do?'
params = {
    'rcpNo': 20170331002299,
    'dcmNo': 5530161,
    'eleId' : 19,
    'offset' : 13730,
    'length' : 78093,
    'dtd' : 'dart3.xsd'
}

In [26]:
resp = requests.get(target_url, params=params)
resp

<Response [200]>

In [27]:
resp.headers

{'execute': 'true', 'Date': 'Sun, 08 Dec 2019 15:26:40 GMT', 'X-XSS-Protection': '1; mode=block', 'Expires': '0', 'Set-Cookie': 'WMONID=5Ivl9IS3jK6; Expires=Tue, 08-Dec-2020 24:26:40 GMT; Path=/, JSESSIONID=ogXAjQi2kFnFMvWAOLTKNwOrwAd8tU1NLzyvmSxLA4BlcyaPyUBpnJdqCwlHYvVM.ZG1fZGFydC9kYXJ0MV9kYXJ0X21zMw==; Path=/; HttpOnly', 'Connection': 'keep-alive', 'Content-Type': 'text/html; charset=utf-8', 'X-Content-Type-Options': 'nosniff', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Transfer-Encoding': 'chunked'}

In [28]:
resp.url
# rcpNo = 날짜+기업코드(6자리) 형식
# dcmNo = ???
# eleId = 19 / offset = 1373 / length = 78093 / dtd = dart3.xsd (고정)
# 다른 기업 예시
# http://dart.fss.or.kr/report/viewer.do?rcpNo=20191202000384&dcmNo=6980773&eleId=18&offset=456065&length=15185&dtd=dart3.xsd
#offset과 length는 0만 아니면 동일하게 실행됨

'http://dart.fss.or.kr/report/viewer.do?rcpNo=20170331002299&dcmNo=5530161&eleId=19&offset=13730&length=78093&dtd=dart3.xsd'